In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
from trino.dbapi import connect
import os

In [ ]:
spark = SparkSession \
    .builder.config("spark.jars", "jars/postgresql-42.5.4.jar") \
    .master("local") \
    .appName("PySpark_Postgres_test") \
    .getOrCreate()

In [ ]:
filename = 'etl_j1'

In [ ]:
conn = connect(
    host="192.168.10.210",
    port=8090,
    user="hive",
    catalog="hive",
    schema=f"{filename}",
)

In [ ]:
tables = pd.read_sql(f"show tables in {filename}", con=conn)
for i,row in tables.iterrows():
    x = row['Table']
    data = pd.read_sql(f"select * from {filename}.{x}", con=conn)
    data.to_csv(f"raw/{x}", index=False)
    data = spark.read.csv(f"file:///HDFS01/airflow/notebooks/Pasit/Database_Connect_Pyspark/raw/{x}",
                            header=True, inferSchema=True)
    data.write \
        .format("jdbc") \
        .mode("overwrite") \
        .option("url", "jdbc:postgresql://192.168.10.22:5432/postgres") \
        .option("driver", "org.postgresql.Driver").option("dbtable", f"postgres.{filename}.{x}") \
        .option("user", "postgres").option("password", "postgres").save()
    os.system(f"rm -f /HDFS01/airflow/notebooks/Pasit/Database_Connect_Pyspark/raw/{x}")
    print("Inserted: ", i,"Value: ", row['Table'])